In [7]:
# Imports:
import os
import sys
import pandas as pd
import shutil
import cv2

# OPTIONS:
save_debug_images=1
box_thickness = 10
box_color = (0, 0, 255)
no_none=0

In [8]:
# Setup Directories:
debug_dir = os.path.join(os.getcwd(),"debug/")
input_dir = os.path.join(os.getcwd(),"data/inputs/")
label_dir = os.path.join(os.getcwd(),"data/labels/")
classes_dir = os.path.join(os.getcwd(),"data/classes.txt")

# Find image and label:
name_of_image=[]
name_of_label=[]
directory = os.listdir(input_dir)

# Setting up classes:
f=open(classes_dir)
count=0
classkey={}
classes={}
for line in f:
    line=line[:-1]
    classkey[line]=[]
    classes[count]=line
    count+=1
if no_none==0:
    classkey["None"]=[]
    classes[count]="None"
box=["Class","x","y","w","h"]

for file in directory:
    if file.endswith((".png","jpg","jpeg")):
        name_of_image.append(file)

In [15]:
for image_name in name_of_image:
# Specify location:
    img_loc = os.path.join(input_dir, image_name)
    if image_name.endswith((".png","jpg")):
        label_loc = os.path.join(label_dir, "{}txt".format(image_name[:-3]))
    elif image_name.endswith(("jpeg")):
        label_loc = os.path.join(label_dir, "{}txt".format(image_name[:-4]))

# Read the original image:
    img = cv2.imread(img_loc,1)
    
# Read the original label:
    try:
        df = pd.read_csv(label_loc, header = None, delimiter=" ", names=box)
    except FileNotFoundError:
        print("{}:Label not found".format(image_name))
        continue
    df['Class'] = df['Class'].map(classes)
    
# Creating Bounding Boxes on image: 
    img_copy=img.copy()
    for i in range(len(df)):
        x=df.iloc[i][1]*img.shape[1]
        y=df.iloc[i][2]*img.shape[0]
        w=df.iloc[i][3]*img.shape[1]
        h=df.iloc[i][4]*img.shape[0]
        start_point = (int(x-(0.5*w)), int(y-(0.5*h)))
        end_point = (int(x+(0.5*w)), int(y+(0.5*h)))
        crop_box=img[start_point[1]:end_point[1],start_point[0]:end_point[0]]
# Save the new image:
        if save_debug_images==1:
            cv2.imwrite(debug_dir+"cropped_{}_{}".format(df.iloc[i][0],image_name), crop_box)